# Training Notebook for a Binary Classification using the Time Windows Approach

### Import Dependencies

In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import os
from utils import *
import json
import random

model_output_path = 'models'
os.makedirs(model_output_path, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
pl.seed_everything(42)
torch.backends.cudnn.deterministic = True

Seed set to 42


### Datasets (k = 12 Folds Cross Validation) & Training of Encoder + MLP

In [ ]:
folds = 12
exp_names_per_fold = 3
labels_to_include = ['ETHANOL', 'ACETONE']
N = 100
hidden_dim = 64
latent_dim = 20
MAX_EPOCHS = 25

df = pd.read_csv("original_training_set_filtered.csv")
df = df[df['label'].isin(labels_to_include)]

exp_labels = df.groupby("exp_name")["label"].first()
label_encoder = LabelEncoder()
label_encoder.fit(exp_labels.values)
exp_name_to_label = exp_labels.to_dict()


exp_names_ACE = list(df[df["label"] == "ACETONE"]["exp_name"].unique())
exp_names_ETH = list(df[df["label"] == "ETHANOL"]["exp_name"].unique())
random.shuffle(exp_names_ACE)
random.shuffle(exp_names_ETH)

splits = {} #splits[fold]["split"] -> list of exp_names of "split"
for fold in range(1, folds + 1):
    splits[fold] = {}
    val_interval = ((fold - 1) * exp_names_per_fold, fold * exp_names_per_fold)
    test_interval = (fold * exp_names_per_fold, (fold + 1) * exp_names_per_fold)
    splits[fold]["val"] = exp_names_ACE[val_interval[0]:val_interval[1]] + exp_names_ETH[val_interval[0]:val_interval[1]]
    splits[fold]["test"] = exp_names_ACE[test_interval[0]:test_interval[1]] + exp_names_ETH[test_interval[0]:test_interval[1]]
    splits[fold]["train"] = list((set(exp_names_ACE) | set(exp_names_ETH)) - set(splits[fold]["val"]) - set(splits[fold]["test"]))


os.makedirs("models/binaryETHvsACE", exist_ok=True)
json.dump(splits, open("models/binaryETHvsACE/fold_splits_exp_names.json", "w"))

for fold in range(1, folds+1):

    # Fold Splitting
    df_train = df[df["exp_name"].isin(splits[fold]["train"])]
    df_val = df[df["exp_name"].isin(splits[fold]["val"])]
    df_test = df[df["exp_name"].isin(splits[fold]["test"])]

    feature_columns = [col for col in df_train.columns if col not in ["label", "exp_name"]]
    scaler = StandardScaler()
    scaler.fit(df_train[feature_columns])
    mean = scaler.mean_
    std = scaler.scale_

    train_dataset = SensorDataset(df = df_train,
                                  label_encoder=label_encoder,
                                  exp_name_to_label=exp_name_to_label,
                                  mean=mean,
                                  std=std,
                                  N=N)
    
    val_dataset = SensorDataset(df = df_val,
                                label_encoder=label_encoder,
                                exp_name_to_label=exp_name_to_label,
                                mean=mean,
                                std=std,
                                N=N)
    
    test_dataset = SensorDataset(df = df_test,
                                 label_encoder=label_encoder,
                                 exp_name_to_label=exp_name_to_label,
                                 mean=mean,
                                 std=std,
                                 N=N)
    
    print(f"\nFold {fold}/{folds} -> Train: {len(train_dataset)} (exp: {len(df_train['exp_name'].unique())}), ",
        f"Val: {len(val_dataset)} (exp: {len(df_val['exp_name'].unique())}), ",
        f"Test: {len(test_dataset)} (exp: {len(df_test['exp_name'].unique())})")
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, persistent_workers=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)

    model = EncoderMLPClassifier(
        num_features=len(feature_columns),
        window_size=N,
        latent_dim=latent_dim,
        hidden_dim=hidden_dim,
        num_classes=2,
        lr=1e-4,
        dropout=0.2,
        weight_decay=1e-5
    )

    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        patience=5,
        verbose=False,
        mode="min"
    )

    trainer = Trainer(
        max_epochs=MAX_EPOCHS,
        callbacks=[early_stop_callback],
        enable_model_summary=False,  # <- Hides the model summary
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1
    )

    trainer.fit(model, train_loader, val_loader)
    # Save the model
    output_name = f"binaryETHvsACE/binaryETHvsACE_EMLP_h{hidden_dim}_l{latent_dim}_{fold}.ckpt"
    model_path = os.path.join(model_output_path, output_name)
    trainer.save_checkpoint(model_path)

    # Test the model
    trainer.test(model, test_loader)

    _, train_exp_stats = get_stats(model_path, train_loader, len(label_encoder.classes_), device)
    train_performance, _, _ = compute_experiment_performance(train_exp_stats)
    print(f"Train Performance: {train_performance}")

    _, val_exp_stats = get_stats(model_path, val_loader, len(label_encoder.classes_), device)
    val_performance, _, _ = compute_experiment_performance(val_exp_stats)
    print(f"Validation Performance: {val_performance}")

    _, test_exp_stats = get_stats(model_path, test_loader, len(label_encoder.classes_), device)
    test_performance, _, _ = compute_experiment_performance(test_exp_stats)
    print(f"Test Performance: {test_performance}")
    

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\miche\OneDrive\Desktop\tesi\repo-tesi\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Fold 1/12 -> Train: 54815 (exp: 61),  Val: 6157 (exp: 6),  Test: 5372 (exp: 6)
Epoch 19: 100%|██████████| 1713/1713 [00:53<00:00, 31.78it/s, v_num=0, train_loss=0.167, train_acc=0.935, val_loss=0.100, val_acc=0.968]   


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 168/168 [00:03<00:00, 52.27it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7109084129333496
        test_loss           1.0299490690231323
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


100%|██████████| 1713/1713 [01:09<00:00, 24.65it/s]


...

100%|██████████| 1694/1694 [01:08<00:00, 24.56it/s]


Train Performance: 0.9672131147540983


100%|██████████| 179/179 [00:46<00:00,  3.86it/s]


Validation Performance: 0.6666666666666666


  0%|          | 0/202 [00:00<?, ?it/s]

In [5]:
last_breakpoint_fold = 11
folds = 12
exp_names_per_fold = 3
labels_to_include = ['ETHANOL', 'ACETONE']
N = 100
hidden_dim = 64
latent_dim = 20
MAX_EPOCHS = 25

df = pd.read_csv("original_training_set_filtered.csv")
df = df[df['label'].isin(labels_to_include)]

exp_labels = df.groupby("exp_name")["label"].first()
label_encoder = LabelEncoder()
label_encoder.fit(exp_labels.values)
exp_name_to_label = exp_labels.to_dict()

#load the splits
splits = json.load(open("models/binaryETHvsACE/fold_splits_exp_names.json", "r"))

for fold in range(last_breakpoint_fold, folds+1):
    fold = str(fold)
    # Fold Splitting
    df_train = df[df["exp_name"].isin(splits[fold]["train"])]
    df_val = df[df["exp_name"].isin(splits[fold]["val"])]
    df_test = df[df["exp_name"].isin(splits[fold]["test"])]

    feature_columns = [col for col in df_train.columns if col not in ["label", "exp_name"]]
    scaler = StandardScaler()
    scaler.fit(df_train[feature_columns])
    mean = scaler.mean_
    std = scaler.scale_

    train_dataset = SensorDataset(df = df_train,
                                  label_encoder=label_encoder,
                                  exp_name_to_label=exp_name_to_label,
                                  mean=mean,
                                  std=std,
                                  N=N)
    
    val_dataset = SensorDataset(df = df_val,
                                label_encoder=label_encoder,
                                exp_name_to_label=exp_name_to_label,
                                mean=mean,
                                std=std,
                                N=N)
    
    test_dataset = SensorDataset(df = df_test,
                                 label_encoder=label_encoder,
                                 exp_name_to_label=exp_name_to_label,
                                 mean=mean,
                                 std=std,
                                 N=N)
    
    print(f"\nFold {fold}/{folds} -> Train: {len(train_dataset)} (exp: {len(df_train['exp_name'].unique())}), ",
        f"Val: {len(val_dataset)} (exp: {len(df_val['exp_name'].unique())}), ",
        f"Test: {len(test_dataset)} (exp: {len(df_test['exp_name'].unique())})")
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, persistent_workers=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)

    model = EncoderMLPClassifier(
        num_features=len(feature_columns),
        window_size=N,
        latent_dim=latent_dim,
        hidden_dim=hidden_dim,
        num_classes=2,
        lr=1e-4,
        dropout=0.2,
        weight_decay=1e-5
    )

    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        patience=5,
        verbose=False,
        mode="min"
    )

    trainer = Trainer(
        max_epochs=MAX_EPOCHS,
        callbacks=[early_stop_callback],
        enable_model_summary=False,  # <- Hides the model summary
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1
    )

    trainer.fit(model, train_loader, val_loader)
    # Save the model
    output_name = f"binaryETHvsACE/binaryETHvsACE_EMLP_h{hidden_dim}_l{latent_dim}_{fold}.ckpt"
    model_path = os.path.join(model_output_path, output_name)
    trainer.save_checkpoint(model_path)

    # Test the model
    trainer.test(model, test_loader)

    _, train_exp_stats = get_stats(model_path, train_loader, len(label_encoder.classes_), device)
    train_performance, _, _ = compute_experiment_performance(train_exp_stats)
    print(f"Train Performance: {train_performance}")

    _, val_exp_stats = get_stats(model_path, val_loader, len(label_encoder.classes_), device)
    val_performance, _, _ = compute_experiment_performance(val_exp_stats)
    print(f"Validation Performance: {val_performance}")

    _, test_exp_stats = get_stats(model_path, test_loader, len(label_encoder.classes_), device)
    test_performance, _, _ = compute_experiment_performance(test_exp_stats)
    print(f"Test Performance: {test_performance}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\miche\OneDrive\Desktop\tesi\repo-tesi\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Fold 11/12 -> Train: 54184 (exp: 61),  Val: 5717 (exp: 6),  Test: 6443 (exp: 6)
Epoch 5: 100%|██████████| 1694/1694 [00:33<00:00, 50.69it/s, v_num=10, train_loss=0.157, train_acc=0.875, val_loss=3.320, val_acc=0.679] 


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 202/202 [00:02<00:00, 82.94it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9312431812286377
        test_loss           0.19805903732776642
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


100%|██████████| 1694/1694 [00:49<00:00, 34.48it/s] 


...

Train Performance: 1.0


100%|██████████| 202/202 [01:07<00:00,  2.98it/s]


Validation Performance: 1.0


100%|██████████| 28/28 [01:06<00:00,  2.37s/it]

Test Performance: 1.0
